In [2]:
pip install torch torchvision faiss-cpu openai sentence-transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [6]:
pip install PyMuPDF langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [ ]:
import gradio as gr
import faiss
import numpy as np
import fitz  # PyMuPDF for PDF text extraction
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory  # Simpler memory
from langchain.text_splitter import RecursiveCharacterTextSplitter
import getpass


# ✅ Load Sentence Transformer for Embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedding_dim = 384

# ✅ FAISS Vector Store
index = faiss.IndexFlatL2(embedding_dim)
stored_texts = []

# ✅ LLM Model (OpenRouter)
API_KEY = getpass.getpass("Enter your OpenRouter API key: ")
BASE_URL = "https://openrouter.ai/api/v1"

llm = ChatOpenAI(
    model="mistralai/mistral-small",
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

# ✅ Memory for Context
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query")

# Clear memory after every 5 queries to refresh context
if len(memory.load_memory_variables({})['chat_history']) > 5:
    memory.clear()


# ✅ Convert Text to Embeddings
def embed_text(text):
    return embedder.encode([text])[0]

# ✅ Add Text to FAISS
def add_to_index(text):
    vector = embed_text(text)
    index.add(np.array([vector], dtype=np.float32))
    stored_texts.append(text)

# ✅ Retrieve Similar Text from PDFs
def retrieve_similar_text(query, top_k=3):
    if len(stored_texts) == 0:
        return ["No relevant data found."]

    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    return [stored_texts[i] for i in indices[0] if i < len(stored_texts)]

# ✅ Create Prompt Template
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    You are an AI assistant answering questions based on uploaded PDFs.
    Context: {context}
    Question: {query}
    Answer:
    """
)

# ✅ Create LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# ✅ Function to Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# ✅ Function to Upload PDF and Store Embeddings
def process_pdf(file):
    pdf_text = extract_text_from_pdf(file)

    # ✅ Improved Chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=50
    )
    chunks = text_splitter.split_text(pdf_text)

    for chunk in chunks:
        add_to_index(chunk)

    return "PDF processed successfully! Now you can ask questions."

# ✅ Generate Response
def generate_response(query):
    contexts = retrieve_similar_text(query)
    context_str = "\n\n---\n\n".join(contexts) if contexts else "No relevant data found."

    response = llm_chain.invoke({"context": context_str, "query": query})

    return f"<pre>{response['text']}</pre>"  # Preserves formatting




# ✅ Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 📄 PDF-Based Chatbot")

    file_upload = gr.File(label="Upload PDF", type="filepath")

    upload_btn = gr.Button("Process PDF")
    output_text = gr.Textbox(label="PDF Processing Status", interactive=False)

    query_input = gr.Textbox(label="Ask a question")
    query_btn = gr.Button("Submit")
    answer_output = gr.Textbox(label="Answer", interactive=False)

    upload_btn.click(process_pdf, inputs=file_upload, outputs=output_text)
    query_btn.click(generate_response, inputs=query_input, outputs=answer_output)

demo.launch(debug=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-7-6268fc45cd57>:26: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
<ipython-input-7-6268fc45cd57>:33: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="query")
<ipython-input-7-6268fc45cd57>:71: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e57f2df9c6d1583eff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio faiss-cpu numpy pymupdf sentence-transformers langchain_community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


**Medical Report**

In [ ]:
import gradio as gr
import faiss
import numpy as np
import fitz  # PyMuPDF for PDF text extraction
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# ✅ Load Sentence Transformer for Medical Embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedding_dim = 384

# ✅ FAISS Vector Store
index = faiss.IndexFlatL2(embedding_dim)
stored_texts = []

# ✅ LLM Model (OpenRouter) - Add Your API Key
API_KEY = getpass.getpass("Enter your OpenRouter API key: ")  # Replace with your actual key
BASE_URL = "https://openrouter.ai/api/v1"


from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="mistralai/mistral-small",
    openai_api_base=BASE_URL,
    openai_api_key=API_KEY,
    request_timeout=60,
    default_headers={"Authorization": f"Bearer {API_KEY}"}  # ✅ OpenRouter authentication
)


# ✅ Memory for Context
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query")

# ✅ Convert Text to Embeddings
def embed_text(text):
    return embedder.encode([text])[0]

# ✅ Add Text to FAISS
def add_to_index(text):
    vector = embed_text(text)
    index.add(np.array([vector], dtype=np.float32))
    stored_texts.append(text)

# ✅ Retrieve Similar Text from Medical Reports
def retrieve_similar_text(query, top_k=3):
    if len(stored_texts) == 0:
        return ["No relevant data found."]

    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    return [stored_texts[i] for i in indices[0] if i < len(stored_texts)]

# ✅ Create Medical Chatbot Prompt
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    You are a medical AI assistant analyzing a patient's medical report.
    Based on the provided medical data, answer the user's query accurately.

    Medical Report Context:
    {context}

    Patient Query:
    {query}

    Medical AI Response:
    """
)

# ✅ Create LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# ✅ Function to Extract Text from Medical Reports
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# ✅ Function to Upload and Process Medical Reports
def process_pdf(file):
    pdf_text = extract_text_from_pdf(file)

    # ✅ Chunking for Better Processing
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=50
    )
    chunks = text_splitter.split_text(pdf_text)

    for chunk in chunks:
        add_to_index(chunk)

    return "Medical report processed successfully! Now you can ask questions."

# ✅ Generate Response
def generate_response(query):
    contexts = retrieve_similar_text(query)  # Retrieve relevant medical info
    context_str = "\n\n---\n\n".join(contexts) if contexts else "No relevant data found."

    response = llm_chain.invoke({"context": context_str, "query": query})
    return response

# ✅ Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🏥 AI Healthcare Chatbot")

    file_upload = gr.File(label="Upload Medical Report (PDF)", type="filepath")
    upload_btn = gr.Button("Process Report")
    output_text = gr.Textbox(label="Processing Status", interactive=False)

    query_input = gr.Textbox(label="Ask a Medical Question")
    query_btn = gr.Button("Submit")
    answer_output = gr.Textbox(label="AI Medical Response", interactive=False)

    upload_btn.click(process_pdf, inputs=file_upload, outputs=output_text)
    query_btn.click(generate_response, inputs=query_input, outputs=answer_output)

demo.launch(debug=True)


Enter your OpenRouter API key: ··········
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6c8fe71a82ec419d8b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6c8fe71a82ec419d8b.gradio.live
